# Installing and Importing required libraries

In [ ]:
!pip -q install h5py
!pip -q install typing-extensions
!pip -q install wheel
!pip -q install imgaug==0.2.5
!pip -q install malaya[gpu]==4.9.0
!pip -q install tensorflow==2.9.0
!pip -q install tensorflow_addons
!pip -q install deep-translator

In [ ]:
import malaya
from random import *
import nltk
import tensorflow as tf
import pandas as pd
from nltk import word_tokenize
from deep_translator import GoogleTranslator
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')

In [ ]:
model = malaya.pos.transformer(model = 'bert')
alignment_en_ms = malaya.alignment.en_ms.huggingface()
alignment_ms_en = malaya.alignment.ms_en.huggingface()

## Importing the required data from real Codeswitch data

In [ ]:
df1 = pd.read_excel('Left_Right_POS_English.xlsx')
df2 = pd.read_excel('Left_Right_POS_Malay.xlsx')

In [ ]:
msdf = pd.read_csv("Malay_Substitutes.csv")
msdf.columns = ["Index", "Standard words", "Accepted words"]
standard_words = list(msdf["Standard words"])
accepted_words = list(msdf["Accepted words"])

In [ ]:
eng_mal = dict()
for i in range(int(df1.index[-1])+1):
    eng_mal[df1.at[i,"POS || POS"]] =  df1.at[i,"English to Malay Conversion Ratio"]

mal_eng = dict()
for i in range(int(df2.index[-1])+1):
    mal_eng[df2.at[i,"POS || POS"]] =  df2.at[i,"Malay to English Conversion Ratio"]

## Converting Monolingual Malay sentences to Code-switch English-Malay Sentence

In [ ]:
mal_noun = ["NOUN","PROPN"]
mal_pron = ["PRON"]
mal_verb = ["ADX","VERB"]
mal_adv = ["ADV"]
mal_adj = ["ADJ"]
mal_adp = ["ADP"]
mal_conj = ["CCONJ","SCONJ"]
mal_det = ["DET"]
mal_par = ["PART"]
mal_num = ["NUM"]
mal_sym = ["SYM"]
mal_oth = ["X", "PUNCT"]

def malPOS(pos_mal):
    mal = ""
    if pos_mal in mal_noun:
        mal = "NOUN"
    elif pos_mal in mal_pron:
        mal = "PRON"
    elif pos_mal in mal_verb:
        mal = "VERB"
    elif pos_mal in mal_adv:
        mal = "ADVB"
    elif pos_mal in mal_adj:
        mal = "ADJC"
    elif pos_mal in mal_adp:
        mal = "ADPN"
    elif pos_mal in mal_conj:
        mal = "CONJ"
    elif pos_mal in mal_det:
        mal = "DETR"
    elif pos_mal in mal_par:
        mal = "PCLE"
    elif pos_mal in mal_num:
        mal = "NMBR"
    elif pos_mal in mal_sym:
        mal = "SYMB"
    else:
        mal = "OTHR"
    
    return mal

In [ ]:
from random import *
def malay_translation_alignment(string_eng, string_mal, alignment_model):
    resulting_model = model.predict(string_mal)
    left_splitted = string_mal.split()
    right_splitted = string_eng.split()
    codeswitch_malay_sentence = ""
    left_flag = False # False --> Malay and True --> English word
    arr = [] # For storing the index which we have already added
    if len(resulting_model) == 1:
        txt = resulting_model[0][0]
        if txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
            codeswitch_malay_sentence = txt
        elif txt.isnumeric():
            codeswitch_malay_sentence = txt
        else:
            if randint(0, 100) > 50:
                codeswitch_malay_sentence = GoogleTranslator(source='ms', target='en').translate(txt)
            else:
                codeswitch_malay_sentence = txt
    else:
        for i in range(len(resulting_model)-1):
              left_txt = resulting_model[i][0]
              left_pos = resulting_model[i][1]
              if i == 0:
                  if left_txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
                      codeswitch_malay_sentence = ""
                  elif left_txt.isnumeric():
                      codeswitch_malay_sentence = left_txt + " "
                  else:
                      if randint(0, 100) > 50:
                          codeswitch_malay_sentence = GoogleTranslator(source='ms', target='en').translate(left_txt)+ " "
                      else:
                          codeswitch_malay_sentence = left_txt+ " "
                  
              right_txt = resulting_model[i+1][0]
              right_pos = resulting_model[i+1][1]
              tag = malPOS(left_pos) + " || " + malPOS(right_pos)
              if left_flag == False:
                  if tag in mal_eng.keys():
                      threshold = mal_eng[tag]
                  else:
                      threshold = 0
              else:
                  if tag in eng_mal.keys():
                      threshold = eng_mal[tag]
                  else:
                      threshold = 0
              rand = randint(0, 100)
              if right_txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
                  codeswitch_malay_sentence = codeswitch_malay_sentence[:-1] + right_txt + " "
              elif right_txt.isnumeric():
                  codeswitch_malay_sentence += right_txt + " "
              else:
                  if rand > threshold:
                      left_flag = False
                      wrd = GoogleTranslator(source='ms', target='en').translate(right_txt)
                      wrd_eng = word_tokenize(wrd)
                      resulting_model_eng = nltk.pos_tag(wrd_eng)
                      if engPOS(resulting_model_eng[0][1]) == malPOS(right_pos):
                          codeswitch_malay_sentence +=  wrd + ' '
                      else:
                          lst = ""
                          for k in alignment_model:
                              if (k[0] == i+1) and not (k[1] in arr):
                                  arr.append(k[1])
                                  lst += right_splitted[k[1]] + " "
                          if len(lst.split()) != 0: 
                              codeswitch_malay_sentence +=  lst[:-1] + ' '
                          else:
                              codeswitch_malay_sentence +=  wrd + ' '
                  else:
                      left_flag = True
                      codeswitch_malay_sentence += right_txt + " "
    return codeswitch_malay_sentence

In [ ]:
malay_text = []
with open('Malay_Normalized.txt') as f:
    malay_text = f.readlines()
f.close()

english_text = []
with open('English_Normalized.txt') as f:
    english_text = f.readlines()
f.close()

resulting_alignment_model = []
size = 200
for i in range(0,len(malay_text),size):
    resulting_alignment_model += alignment_ms_en.align(malay_text[i:i+size], english_text[i:i+size], align_layer = 7)


In [ ]:
malay_cs = []
for i in range(len(malay_text)):
    string_mal = malay_text[i]
    string_eng = english_text[i]
    alignment_model = resulting_alignment_model[i]
    codeswitch_malay_sentence = malay_translation_alignment(string_eng, string_mal, alignment_model)
    malay_cs.append(codeswitch_malay_sentence)


In [ ]:
malay_cs_arr = []
for i in malay_cs:
        txt = ""
        for j in range(len(standard_words)):
            txt = i.replace(standard_words[j], accepted_words[j])
        t = txt.split()
        malay_cs_arr.append(" ".join(t))

malay_cs_text = ""
for i in malay_cs_arr:
    malay_cs_text += i + '\n'

f1 = open('Malay_CS.txt','w+')
f1.write(malay_cs_text)
f1.close()

## Converting Monolingual English sentences to Code-switch English-Malay Sentence

In [ ]:
eng_noun = ["NN","NNP","NNPS","NNS"]
eng_pron = ["PRP","PRP$","WP","WP$"]
eng_verb = ["VB","VBD","VBG","VBN","VBP","VBZ","MD"]
eng_adv = ["RB","RBR","RBS"]
eng_adj = ["JJ","JJR","JJS"]
eng_adp = ["IN"]
eng_conj = ["CC"]
eng_det = ["DT","PDT","TO","EX"]
eng_par = ["RP"]
eng_num = ["CD"]
eng_sym = ["$","(",")",",","--",".",":","SYM","``","''"]
eng_oth = ["FW","LS","POS","UH"]

def engPOS(pos_eng):
    eng = ""
    if pos_eng in eng_noun:
        eng = "NOUN"
    elif pos_eng in eng_pron:
        eng = "PRON"
    elif pos_eng in eng_verb:
        eng = "VERB"
    elif pos_eng in eng_adv:
        eng = "ADVB"
    elif pos_eng in eng_adj:
        eng = "ADJC"
    elif pos_eng in eng_adp:
        eng = "ADPN"
    elif pos_eng in eng_conj:
        eng = "CONJ"
    elif pos_eng in eng_det:
        eng = "DETR"
    elif pos_eng in eng_par:
        eng = "PCLE"
    elif pos_eng in eng_num:
        eng = "NMBR"
    elif pos_eng in eng_sym:
        eng = "SYMB"
    else:
        eng = "OTHR"
    
    return eng

In [ ]:
from random import *
def english_translation_alignment(string_eng, string_mal, alignment_model):
    sentence_eng = word_tokenize(string_eng)
    resulting_model = nltk.pos_tag(sentence_eng)    
    right_splitted = string_mal.split()
    left_splitted = string_eng.split()
    codeswitch_english_sentence = ""
    left_flag = True # False --> Malay and True --> English word
    arr = [] # For storing the index which we have already added
    if len(resulting_model) == 1:
        txt = resulting_model[0][0]
        if txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
            codeswitch_english_sentence = txt
        elif txt.isnumeric():
            codeswitch_english_sentence = txt
        else:
            if randint(0, 100) > 50:
                codeswitch_english_sentence = GoogleTranslator(source='en', target='ms').translate(txt)
            else:
                codeswitch_english_sentence = txt
    else:
        for i in range(len(resulting_model)-1):
              left_txt = resulting_model[i][0]
              left_pos = resulting_model[i][1]
              if i == 0:
                  if left_txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
                      codeswitch_english_sentence = ""
                  elif left_txt.isnumeric():
                      codeswitch_english_sentence = left_txt + " "
                  else:
                      if randint(0, 100) > 50:
                          codeswitch_english_sentence = GoogleTranslator(source='en', target='ms').translate(left_txt)+ " "
                      else:
                          codeswitch_english_sentence = left_txt+ " "
                  
              right_txt = resulting_model[i+1][0]
              right_pos = resulting_model[i+1][1]
              tag = engPOS(left_pos) + " || " + engPOS(right_pos)
              if left_flag == False:
                  if tag in mal_eng.keys():
                      threshold = mal_eng[tag]
                  else:
                      threshold = 0
              else:
                  if tag in eng_mal.keys():
                      threshold = eng_mal[tag]
                  else:
                      threshold = 0
              rand = randint(0, 100)
              if right_txt in list("!#$%&'()*+,-./:;<=>?@[\]^_`{|}~"):
                  codeswitch_english_sentence = codeswitch_english_sentence[:-1] + right_txt + " "
              elif right_txt.isnumeric():
                  codeswitch_english_sentence += right_txt + " "
              else:
                  if rand > threshold:
                      left_flag = False
                      wrd = GoogleTranslator(source='en', target='ms').translate(right_txt)
                      resulting_model_mal = model.predict(string_mal)
                      if malPOS(resulting_model_mal[0][1]) == engPOS(right_pos):
                          codeswitch_english_sentence +=  wrd + ' '
                      else:
                          lst = ""
                          for k in alignment_model:
                              if (k[0] == i+1) and not (k[1] in arr):
                                  arr.append(k[1])
                                  lst += right_splitted[k[1]] + " "
                          if len(lst.split()) != 0: 
                              codeswitch_english_sentence +=  lst[:-1] + ' '
                          else:
                              codeswitch_english_sentence +=  wrd + ' '
                  else:
                      left_flag = True
                      codeswitch_english_sentence += right_txt + " "
    return codeswitch_english_sentence

In [ ]:
malay_text = []
with open('Malay_Normalized.txt') as f:
    malay_text = f.readlines()
f.close()

english_text = []
with open('English_Normalized.txt') as f:
    english_text = f.readlines()
f.close()

resulting_alignment_model = []
size = 200
for i in range(0,len(english_text),size):
    resulting_alignment_model += alignment_en_ms.align(english_text[i:i+size], malay_text[i:i+size], align_layer = 7)


In [ ]:
english_cs = []
for i in range(len(english_text)):
    string_mal = malay_text[i]
    string_eng = english_text[i]
    alignment_model = resulting_alignment_model[i]
    codeswitch_english_sentence = english_translation_alignment(string_eng, string_mal, alignment_model)
    english_cs.append(codeswitch_english_sentence)


In [ ]:
english_cs_arr = []
for i in english_cs:
        txt = ""
        for j in range(len(standard_words)):
            txt = i.replace(standard_words[j], accepted_words[j])
        t = txt.split()
        english_cs_arr.append(" ".join(t))

english_cs_text = ""
for i in english_cs_arr:
    english_cs_text += i + '\n'

f1 = open('English_CS.txt','w+')
f1.write(english_cs_text)
f1.close()